In [1]:
"""Libraries"""

%matplotlib inline
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from ipywidgets import interact
import numpy as np 
from scipy.integrate import solve_ivp
from scipy.signal import argrelextrema
import time
from numba import jit, njit, prange, float64, float32, int64, int32
from numba.experimental import jitclass
import math
import pandas as pd
import importlib
import random

# my own file:
import diffeq as de
importlib.reload(de)

class dotdict(dict):
    #dot.notation access to dictionary attributes
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [2]:
def function(x, y):
    return 0.015*np.exp(-0.03*(x**2 + y**2))* ((x**2+y-11)**2 + (x+y**2-7)**2) - 0.1*x - 0.05*y + 1.0

def trail(X, Y, delta=0.1):
    Z = []
    for x, y in zip(X, Y):
        Z.append(function(x, y) + delta)
    return Z

def surface(X, Y):
    Z = []
    for y in Y:
        z = []
        for x in X:
            z.append(function(x, y))
        Z.append(z)
    return Z

def rand_point(ranges):
    point = []
    for key in ranges:
        point.append(random.uniform(ranges[key][0], ranges[key][1]))
    return point

def in_range(point, ranges):
    for i, key in enumerate(ranges):
        if point[i] < ranges[key][0] or ranges[key][1] < point[i]:
            return False
    return True

def minimal(points):
    best = points[0]
    for point in points:
        if point[-1] < best[-1]:
            best = point
    return best

## Pattern search

In [90]:
ranges = dotdict(dict(
    x=[-6.0, 6.0],
    y=[-6.0, 6.0],
))

In [91]:
def pattern_search(ranges, function, start_point, max_steps=100, decay=0.8, min_step=0.005, first_step=0.1, stop_condition=0.004):
    last_point = start_point#rand_point(ranges)
    last_point.append(function(last_point[0], last_point[1]))
    best_points = [last_point]
    point_num = 0
    step_num = 0
    loc_min = False
    while len(best_points) < max_steps and not loc_min:
        new_points = []
        step_size = 0.0
        for i, key in enumerate(ranges):
            delta = abs(ranges[key][1] - ranges[key][0])
            delta = min_step*delta + first_step*delta*decay**step_num
            
            new_point = last_point[:-1]
            new_point[i] = last_point[i] - delta
            if in_range(new_point, ranges):
                new_point.append(function(new_point[0], new_point[1]))
                new_points.append(new_point)
                point_num += 1
                
            new_point = last_point[:-1]
            new_point[i] = last_point[i] + delta
            if in_range(new_point, ranges):
                new_point.append(function(new_point[0], new_point[1]))
                new_points.append(new_point)
                point_num += 1
                
        last_point = minimal(new_points)
        best_points.append(last_point)
        step_num += 1
        
        if step_num > 10:
            if abs(best_points[-2][-1] - best_points[-1][-1]) < stop_condition * best_points[-1][-1] and abs(best_points[-4][-1] - best_points[-3][-1]) < stop_condition * best_points[-1][-1]:
                loc_min = True
    print(f'{point_num=};   {step_num=}')
    print(f'{best_points[-4][-1]: .4f}; {best_points[-3][-1]: .4f}; {best_points[-2][-1]: .4f}; {best_points[-1][-1]: .4f}')
    return best_points

In [99]:
all_points = []
all_points += pattern_search(ranges, function, [0, 0.5])
X1 = [point[0] for point in all_points]
Y1 = [point[1] for point in all_points]
Z1 = [point[2]+0.1 for point in all_points]
all_points = []
all_points += pattern_search(ranges, function, [-1, -5])
X2 = [point[0] for point in all_points]
Y2 = [point[1] for point in all_points]
Z2 = [point[2]+0.1 for point in all_points]
all_points = []
all_points += pattern_search(ranges, function, [-4.5, 1.5])
X3 = [point[0] for point in all_points]
Y3 = [point[1] for point in all_points]
Z3 = [point[2]+0.1 for point in all_points]
all_points = []
all_points += pattern_search(ranges, function, [1.5, -4])
X4 = [point[0] for point in all_points]
Y4 = [point[1] for point in all_points]
Z4 = [point[2]+0.1 for point in all_points]
all_points = []

point_num=52;   step_num=13
 0.5953;  0.5933;  0.5942;  0.5929
point_num=55;   step_num=14
 1.5318;  1.5376;  1.5316;  1.5355
point_num=60;   step_num=15
 1.1135;  1.1173;  1.1134;  1.1165
point_num=56;   step_num=14
 0.7288;  0.7291;  0.7274;  0.7277


In [111]:
size = 6.0
res = 50
x = np.linspace(-size, size, res)
y = np.linspace(-size, size, res)
z = surface(x, y)

layout = go.Layout(
    autosize=False,
    width=800, height=600,
    showlegend=False,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=75,
        pad = 4
    ),
)

fig = px.scatter_3d(
    x = [], y = [], z = [],
)
fig.update_traces(marker=dict(
    size=3,
    color='crimson',
))
fig.add_traces(
    go.Surface(
        x = x, y = y, z = z,
        colorscale='turbo',
))
fig.add_traces(
    go.Scatter3d(
        x = X1, y = Y1, z = Z1,
        marker=dict(
            size=3,
            color='crimson',
        ),
        line=dict(
            color='crimson',
            width=2
        )
))
fig.add_traces(
    go.Scatter3d(
        x = X2, y = Y2, z = Z2,
        marker=dict(
            size=3,
            color='blue',
        ),
        line=dict(
            color='blue',
            width=2
        )
))
fig.add_traces(
    go.Scatter3d(
        x = X3, y = Y3, z = Z3,
        marker=dict(
            size=3,
            color='green',
        ),
        line=dict(
            color='green',
            width=2
        )
))
fig.add_traces(
    go.Scatter3d(
        x = X4, y = Y4, z = Z4,
        marker=dict(
            size=3,
            color='orange',
        ),
        line=dict(
            color='orange',
            width=2
        )
))
fig.update_layout(layout)
fig.show()

In [53]:
print(f'smallest point: {np.min(surface(x, y))}')
print(f'with patter search: {min([point[2] for point in all_points])}')
print(f'number of points: {len(all_points)}')

smallest point: 0.5933808473062121
with patter search: 0.5928509188809544
number of points: 60


## Bruteforce parameter sweep

In [112]:
size = 6.0
res = 50
x = np.linspace(-size, size, res)
y = np.linspace(-size, size, res)
z = surface(x, y)
X = []; Y = []; Z = []
for x_ in np.linspace(-size, size, 30):
    for y_ in np.linspace(-size, size, 30):
        X.append(x_)
        Y.append(y_)
        Z.append(function(x_, y_)+0.05)

layout = go.Layout(
    autosize=False,
    width=800, height=600,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=75,
        pad = 4
    ),
)

fig = px.scatter_3d(
    x = X, y = Y, z = Z,
)
fig.update_traces(marker=dict(
    size=2,
    color='crimson',
))
fig.add_traces(
    go.Surface(
        x = x, y = y, z = z,
        colorscale='turbo',
))
fig.update_layout(layout)
fig.show()

## Adaptive parameter study

In [113]:
axis_names = ['x', 'y']

grids = [
    dotdict(dict(
        x=[-6.0, 6.0, 10],
        y=[-6.0, 6.0, 10],
    )),
    dotdict(dict(
        x=[-6.0, 6.0, 40],
        y=[-6.0, 6.0, 40],
        num=5
    ))
]

search = []

for n, grid in enumerate(grids):
    ranges = dotdict(dict(    # stores possible values, an axis can take
        x=[],
        y=[],
    ))
    points = dotdict(dict(
        points=[],
        x=[],
        y=[],
        z=[],
    ))
# fill ranges
    for axis in axis_names:
        div = grid[axis]
        if len(div) == 3:
            ranges[axis] = np.linspace(div[0], div[1], div[2])
            grid[axis].append(ranges[axis][1] - ranges[axis][0])    # step_size
        else:
            ranges[axis] = [div[0]]
            grid[axis].append(None)    # no step in this direction
            
# first run, bruteforce parameter sweep
    if n==0:
        for x in ranges.x:
            for y in ranges.y:
                if [x, y] not in points.points:
                    points.points.append([x, y])
# adaptive search around best                    
    else:
        for point in search[-1].points[:grid.num]:   # best num points from last run
            loc_range = dotdict(dict(
                x=[],
                y=[],
            ))
            for i, axis in enumerate(axis_names):
                last_step_size = grids[n-1][axis][-1]
                if last_step_size == None:
                    loc_range[axis] = ranges[axis]
                else:
                    start = max(grid[axis][0], point[i] - last_step_size)
                    end = min(grid[axis][1], point[i] + last_step_size)
                    for value in ranges[axis]:
                        if start < value and value < end:
                            loc_range[axis].append(value)
                            
            for x in loc_range.x:
                for y in loc_range.y:
                    if [x, y] not in points.points:
                        points.points.append([x, y])
# calculate new points                
    points.x = np.array([x for [x, y] in points.points])
    points.y = np.array([y for [x, y] in points.points])
    points.z = function(points.x, points.y)
    for i, z in enumerate(points.z):
        points.points[i].append(z)
# sort points by z        
    points.points.sort(key=lambda point : point[-1])
# save points
    search.append(points)

In [116]:
size = 6.0
res = 50
x = np.linspace(-size, size, res)
y = np.linspace(-size, size, res)
z = surface(x, y)

layout = go.Layout(
    autosize=False,
    width=800, height=600,
    showlegend=False,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=75,
        pad = 4
    ),
)

fig = px.scatter_3d(
    x = search[0].x, y = search[0].y, z = search[0].z + 0.05,
)
fig.update_traces(marker=dict(
    size=3,
    color='royalblue',
    line=dict(width=8,
        color='white'),
))

fig.add_trace(
    go.Scatter3d(
        x = search[1].x, y = search[1].y, z = search[1].z + 0.05,
        mode='markers',
        marker=dict(
            size=2,
            color='crimson',
        )
))

fig.add_traces(
    go.Surface(
        x = x, y = y, z = z,
        colorscale='turbo',
))

fig.update_layout(layout)
fig.show()

print(len(search[0].points) + len(search[1].points))

334


## Adaptive search mark 2

In [117]:
axis_names = ['x', 'y']

grids = [
    dotdict(dict(
        x=dotdict(dict(start=-6.0, end=6.0, division=10)),
        y=dotdict(dict(start=-6.0, end=6.0, division=10)),
    )),
    dotdict(dict(
        x=dotdict(dict(start=-6.0, end=6.0, division=40)),
        y=dotdict(dict(start=-6.0, end=6.0, division=40)),
        num=5
    ))
]

for i in range(len(grids)):
    for axis in axis_names:
        grids[i][axis].range = np.linspace(grids[i][axis].start, grids[i][axis].end, grids[i][axis].division)
        if grids[i][axis].division == 1:
            grids[i][axis].step = None
        else:
            grids[i][axis].step = grids[i][axis].range[1] - grids[i][axis].range[0]
        
    all_points = []    
    for x in grids[i].x.range:
        for y in grids[i].y.range:
            all_points.append([x, y])
            
    grids[i].all_points = all_points

In [118]:
points = []
for point in grids[0].all_points:
    points.append([point[0], point[1], function(point[0], point[1])])
points.sort(key=lambda point : point[-1])
grids[0].points = points

for i in range(1, len(grids)):
    to_eval = []
    for good_point in grids[i-1].points[:grids[i].num]:
        all_points_copy = [point for point in grids[i].all_points]
        for point in all_points_copy:
            close = True
            for j, axis in enumerate(axis_names):
                if grids[i-1][axis].step != None:
                    close &= abs(point[j] - good_point[j]) < grids[i-1][axis].step
            if close:
                grids[i].all_points.remove(point)
                to_eval.append(point)
            
    points = []
    for point in to_eval:
        points.append([point[0], point[1], function(point[0], point[1])])
    points.sort(key=lambda point : point[-1])
    grids[i].points = points

In [120]:
size = 6.0
res = 50
X = np.linspace(-size, size, res)
Y = np.linspace(-size, size, res)
Z = surface(X, Y)

layout = go.Layout(
    autosize=False,
    width=800, height=600,
    showlegend=False,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=75,
        pad = 4
    ),
)

x = np.array([x for [x, y, z] in grids[0].points])
y = np.array([y for [x, y, z] in grids[0].points])
z = np.array([z for [x, y, z] in grids[0].points])
fig = px.scatter_3d(
    x = x, y = y, z = z + 0.05,
)
fig.update_traces(marker=dict(
    size=3,
    color='royalblue',
    line=dict(width=8,
        color='white'),
))

x = np.array([x for [x, y, z] in grids[1].points])
y = np.array([y for [x, y, z] in grids[1].points])
z = np.array([z for [x, y, z] in grids[1].points])
fig.add_trace(
    go.Scatter3d(
        x = x, y = y, z = z + 0.05,
        mode='markers',
        marker=dict(
            size=2,
            color='crimson',
        )
))

fig.add_traces(
    go.Surface(
        x = X, y = Y, z = Z,
        colorscale='turbo',
))

fig.update_layout(layout)
fig.show()

print(len(grids[0].points) + len(grids[1].points))

334


## Bayesian optimization

In [123]:
from bayes_opt import BayesianOptimization    # https://github.com/fmfn/BayesianOptimization
from bayes_opt import UtilityFunction

ranges = dotdict(dict(
    x=[-6.0, 6.0],
    y=[-6.0, 6.0],
))

utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)

optimizer = BayesianOptimization(
    f=None,
    pbounds=ranges,
    verbose=2,    # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=0,
)

In [124]:
all_points = []
for i in range(40):
    point = rand_point(ranges)
    value = function(point[0], point[1])
    optimizer.register(
            params=point,
            target=10-value,
    )
    point.append(value)
    all_points.append(point)
    
for i in range(60):
    suggestion = optimizer.suggest(utility)
    point = [suggestion['x'], suggestion['y']]
    value = function(point[0], point[1])
    optimizer.register(
            params=point,
            target=10-value,
    )
    point.append(value)
    all_points.append(point)

In [125]:
X = [point[0] for point in all_points]
Y = [point[1] for point in all_points]
Z = [point[2]+0.1 for point in all_points]

size = 6.0
res = 50
x = np.linspace(-size, size, res)
y = np.linspace(-size, size, res)
z = surface(x, y)

layout = go.Layout(
    autosize=False,
    width=800, height=600,
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=75,
        pad = 4
    ),
)

fig = px.scatter_3d(
    x = X, y = Y, z = Z,
)
fig.update_traces(marker=dict(
    size=3,
    color='crimson',
))
fig.add_traces(
    go.Surface(
        x = x, y = y, z = z,
        colorscale='turbo',
))
fig.update_layout(layout)
fig.show()